# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [1]:
# Trecho de importações
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

# Trecho de leitura dos arquivos a serem utilizados
X_treino = pd.read_csv('X_treino.csv', index_col = [0, 1])
X_teste = pd.read_csv('X_teste.csv', index_col = [0, 1])
y_treino = pd.read_csv('y_treino.csv', sep = '\t')
y_teste = pd.read_csv('y_teste.csv', sep = '\t')

# Impressão dos formatos dos conjuntos
print("Tamanho de X_treino:", X_treino.shape)
print("Tamanho de X_teste:", X_teste.shape)
print("Tamanho de y_treino:", y_treino.shape)
print("Tamanho de y_teste:", y_teste.shape)

Tamanho de X_treino: (7352, 561)
Tamanho de X_teste: (2947, 561)
Tamanho de y_treino: (7352, 1)
Tamanho de y_teste: (2947, 1)


### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [2]:
# Extraindo conjuntos de validação de dentro dos treinamentos
X_train, X_validacao, y_train, y_val = train_test_split(X_treino, y_treino, test_size = 0.25, random_state = 42)

# Impressão dos formatos dos conjuntos
print("Tamanho de X_treino:", X_train.shape)
print("Tamanho de X_validacao:", X_validacao.shape)
print("Tamanho de y_treino:", y_train.shape)
print("Tamanho de y_validacao:", y_val.shape)

Tamanho de X_treino: (5514, 561)
Tamanho de X_validacao: (1838, 561)
Tamanho de y_treino: (5514, 1)
Tamanho de y_validacao: (1838, 1)


### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [3]:
%%time

top_3 = []

# Rodando a árvore requisitada
clf = DecisionTreeClassifier(random_state = 42, max_depth = 4).fit(X_train, y_train)

# Coletando o TOP 3 importancias
importancias = pd.Series(clf.feature_importances_, index = X_train.columns)
importancias.sort_values(ascending = False, inplace = True)
top_3 = importancias.index[:3]

importancias.head(3)

Wall time: 2min 37s


338 fBodyAcc-bandsEnergy()-57,64    0.2
294 fBodyAcc-meanFreq()-X           0.2
154 tBodyGyro-arCoeff()-Z,1         0.2
dtype: float64

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [6]:
%%time

X_train_3 = X_train[top_3]
X_val_3 = X_validacao[top_3]
X_teste_3 = X_teste[top_3]

path = DecisionTreeClassifier(random_state = 42).cost_complexity_pruning_path(X_train_3, y_train)
ccp_alphas = path.ccp_alphas

ccp_alphas = ccp_alphas[::5]

clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state = 42, ccp_alpha = ccp_alpha).fit(X_val_3, y_val)
    clfs.append(clf)

val_scores = [clf.score(X_val_3, y_val) for clf in clfs]

ind_melhor_arvore = len(val_scores) - val_scores[::-1].index(max(val_scores)) - 1
melhor_arvore = clfs[ind_melhor_arvore]

acuracia_valid = melhor_arvore.score(X_val_3, y_val)

print(f'Acurácia: {acuracia_valid * 100: .1f} %')
melhor_arvore

Acurácia:  100.0 %
Wall time: 6min 48s


DecisionTreeClassifier(random_state=42)

### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [7]:
acuracia_teste = melhor_arvore.score(X_teste_3, y_teste)

print(f'Acurácia: {acuracia_teste * 100: .1f} %')

Acurácia:  0.0 %


> Apesar de fazer e refazer a tarefa algumas vezes, não entendo o porquê de obter 100% e 0% de acurácia, respectivamente, nos conjuntos Treino/Validação e no Teste.

> Mesmo entendendo que isso por ser um <i>overfit</i> do ccp_alpha, o Cross-Validation tem o ideal de evitar isso, no entanto, nao consigo descobrir o porquê. Alterei conjuntos utilizados no fit da Árvore, alterei seed, alterei cortes de ccp_alpha...

> Talvez seja um equívoco meu durante a interpretação do Módulo, uma vez que perdi as contas das vezes que o professor menciona "treino" e esta se referindo a "validação", ou algo assim. Sinto que o mesmo deve ter acontecido nos enunciados do exercicio.

> Ainda sim, sinto também que há um certo "vão de aprendizado" entre o proposto em aula e na tarefa, diferentemente do que há na maioria dos módulos anteriores.